In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gensim

import torch
from torch import nn
import gensim
import fasttext
from gensim.models import FastText
import random
import operator
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pickle
import joblib
import nltk

C:\Users\ntaylor\Anaconda3\envs\deeplearning\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df_discharge_train = pd.read_csv("../clinicalBERT/data/discharge/train.csv")
df_discharge_val = pd.read_csv("../clinicalBERT/data/discharge/val.csv")
df_discharge_test = pd.read_csv("../clinicalBERT/data/discharge/test.csv")
df_3day_train = pd.read_csv("../clinicalBERT/data/3days/train.csv")
df_3day_val = pd.read_csv("../clinicalBERT/data/3days/val.csv")
df_3day_test = pd.read_csv("../clinicalBERT/data/3days/test.csv")

In [3]:
# concat em all

dfs = [df_discharge_train,df_discharge_val, df_discharge_test, df_3day_train, df_3day_val, df_3day_test]
df_all = pd.concat(dfs)
df_all.shape

(91353, 4)

In [4]:
training_data = df_all.TEXT.values

In [5]:
training_data.shape

(91353,)

In [78]:
# training_corpus = df_all.TEXT.apply(lambda x: x.split(" "))
# training_corpus

In [11]:
tokenizer = nltk.RegexpTokenizer(r"\w+").tokenize
training_corpus = df_all.TEXT.apply(tokenizer)
model = gensim.models.Word2Vec(sentences=training_corpus, vector_size=200, window=5, min_count=4, workers=4)

In [13]:
model.save("./data/mimic3/new_mimic_word2vec_200_strict.model")

In [81]:
model = gensim.models.KeyedVectors.load("./data/mimic3/new_mimic_word2vec_200_strict.model")

In [12]:
weights = torch.FloatTensor(model.wv.vectors)
print("weights vector shape: ", weights.shape)

weights vector shape:  torch.Size([35786, 200])


In [14]:
model.wv.key_to_index

{'and': 0,
 'the': 1,
 'to': 2,
 'of': 3,
 'with': 4,
 'in': 5,
 'for': 6,
 'no': 7,
 'on': 8,
 'is': 9,
 'was': 10,
 'a': 11,
 '1': 12,
 'mg': 13,
 'pt': 14,
 's': 15,
 'at': 16,
 '2': 17,
 'as': 18,
 'patient': 19,
 'left': 20,
 'right': 21,
 'or': 22,
 'ml': 23,
 'he': 24,
 'po': 25,
 'this': 26,
 '5': 27,
 'am': 28,
 'tablet': 29,
 'not': 30,
 '3': 31,
 'blood': 32,
 'there': 33,
 'pain': 34,
 'from': 35,
 'she': 36,
 'are': 37,
 'c': 38,
 'history': 39,
 '4': 40,
 'p': 41,
 'normal': 42,
 'o': 43,
 'daily': 44,
 'chest': 45,
 't': 46,
 'be': 47,
 'ct': 48,
 'l': 49,
 'has': 50,
 '0': 51,
 'his': 52,
 '6': 53,
 'pm': 54,
 'day': 55,
 'by': 56,
 'w': 57,
 'one': 58,
 'sig': 59,
 'but': 60,
 'her': 61,
 '8': 62,
 'plan': 63,
 'given': 64,
 'hr': 65,
 'had': 66,
 'were': 67,
 'dl': 68,
 '7': 69,
 'status': 70,
 '10': 71,
 'reason': 72,
 'iv': 73,
 'assessment': 74,
 'discharge': 75,
 '9': 76,
 'other': 77,
 'that': 78,
 'medical': 79,
 'noted': 80,
 'medications': 81,
 'acute': 82,
 '

In [19]:
model.wv.key_to_index['.']

KeyError: '.'

In [16]:
sims = model.wv.most_similar('lung', topn=10)  # get other similar words
sims

[('hemithorax', 0.5806155204772949),
 ('airspace', 0.5467953681945801),
 ('midlung', 0.5394905805587769),
 ('lungs', 0.5317095518112183),
 ('lobe', 0.4938564598560333),
 ('patchy', 0.46287256479263306),
 ('incomprehensible', 0.45803302526474),
 ('perihilar', 0.44481945037841797),
 ('breat', 0.44277915358543396),
 ('lingula', 0.43781715631484985)]

In [17]:
model.wv.most_similar('heart', topn=10)

[('cardiac', 0.5015909671783447),
 ('hrt', 0.4965977966785431),
 ('multiorgan', 0.4900466799736023),
 ('thrive', 0.4851595163345337),
 ('repsiratory', 0.4788668155670166),
 ('respriatory', 0.4561050236225128),
 ('biverntricular', 0.4146994352340698),
 ('multisystem', 0.39116209745407104),
 ('ventricular', 0.36805665493011475),
 ('kidney', 0.36537548899650574)]

In [18]:
model.wv.most_similar('leg', topn=10)

[('arm', 0.7791668176651001),
 ('foot', 0.7185063362121582),
 ('ankle', 0.6814638376235962),
 ('elbow', 0.6774156093597412),
 ('forearm', 0.6740267872810364),
 ('thigh', 0.6683149933815002),
 ('hand', 0.6659051179885864),
 ('shoulder', 0.6643431186676025),
 ('thumb', 0.6614435315132141),
 ('heel', 0.6496439576148987)]